In [ ]:
import cv2
import numpy as np
import tqdm
import json
import base64
import requests
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
token = "сюда вставить свой токен из mcs"

HOST = "https://smarty.mail.ru/api/v1/photo/improve"

# input_file = "test.mp4"
output_file = "output.avi"

In [ ]:
def request_backend(img, token):
    files = {}
    name = "file_0"
    files[name] = img
        
    meta = {
            "mode":["improve"],
            "images": [{"name": name}]
           }
                                                              
    data = {'meta': json.dumps(meta)}
    response = requests.post(HOST + "?oauth_provider=mcs&oauth_token={}" \
                      .format(token), data=data, files=files)

    return response

In [ ]:
def from_base64(base64_data):
    nparr = np.fromstring(base64.b64decode(base64_data), np.uint8)
    img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return img
    #return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

def parse_response(response):
    body = response.json()['body']
    improve = body['improve']
    base64_image = improve[0]['colorized_improved']
    img = from_base64(base64_image)
    return img

In [ ]:
cap = cv2.VideoCapture(input_file)

fps = int(cap.get(cv2.CAP_PROP_FPS))
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
num_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

out = cv2.VideoWriter(output_file,
                      cv2.VideoWriter_fourcc('M','J','P','G'),
                      fps,
                      (frame_width, frame_height))

for iFrame in tqdm.tqdm(range(num_frames)):
    ret, frame = cap.read()
    # если цветное видео
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) 
    
    if frame is not None:
        buf = cv2.imencode('.jpg', frame)[1].tostring()
        response = request_backend(buf, token)
        image = parse_response(response)
        
        out.write(image)
        
        # пока, чтобы много денег не тратить!
        if iFrame > 60:
            break

cap.release()
out.release()